# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import csv
import datetime

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [18]:
#ascertain what elements are in citipy library
print(dir(citipy))

['City', 'WORLD_CITIES_DICT', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_current_dir', '_current_filename', '_world_cities_csv_path', '_world_cities_kdtree', 'c', 'cities', 'city', 'city_coordinate_key', 'csv', 'csv_file', 'kdtree', 'nearest_city', 'os']


In [19]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination (citipy-specific functions)
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

617

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [20]:
#URL syntax for OpenWeather api call
ow_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key 
#will need additional URL for loop querying by city id + city

#placeholders for loop retrievals
city_num = []
city_name = []
countries = []
date = []
max_temp = []
humidity = []
windy = []
cloudy = []
latitude=[]
longitude =[]

#using city in citipy cities list, query openweather via api call syntax to obtain and hold weather data for each
for city in cities:
    data = requests.get(ow_url).json()
    #create query url
    ow_query_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key + "&q=" + city
    response = requests.get(ow_query_url).json()
    
    try:  
        city_id = response['id']
        city_num.append(city_id)
        citynm = response['name']
        city_name.append(citynm)
        today_date = response['dt']
        date.append(today_date)
        country_code = response['sys']['country']
        countries.append(country_code)
        temp = response['main']['temp_max']
        max_temp.append(temp)
        hum = response['main']['humidity']
        humidity.append(hum)
        wind = response['wind']['speed']
        windy.append(wind)
        cloud = response['clouds']['all']
        cloudy.append(cloud)
        lat = response['coord']['lat']
        latitude.append(lat)
        lng = response['coord']['lon']
        longitude.append(lng)

    except KeyError:
        print(city_id, city)

6151455 saleaula
6151455 naftah
1507390 mixtepec
4031742 barentsburg
3445487 maneadero
8010472 taolanaro
1792592 illoqqortoormiut
3896218 labutta
4021858 belushya guba
1733782 attawapiskat
2121197 artyk
779622 dinsor
2065176 tabiauea
699986 bargal
2126199 sentyabrskiy
2120047 berbera
3421982 bantry
5924351 barawe
3924895 louisbourg
1513157 maarianhamina
6089404 monatele
4011743 mys shmidta
1529484 vaitupu
2127060 bengkulu
5847411 ijaki
2271968 inderborskiy
7626384 amderma
757718 grand river south east
1655124 tsihombe
4975802 mutsamudu
4975802 barbar
1174872 jibuti
2064550 bentiu
1636308 nizhneyansk
2377457 moyo
2039557 santa eulalia del rio
6320062 toungoo
4220862 sakakah
6301965 ikole
6301965 buqayq
3455051 samusu
6111867 rawah
3838859 sahrak
1067565 tubruq
3427833 pousat
2314705 sorvag
1786657 marcona
5969025 kristiinankaupunki
576317 umzimvubu
576317 kuche
2517750 dzhusaly
3899695 aflu
2164422 longlac
2164422 asau
3220813 palabuhanratu
1683425 krasnoselkup
1683425 kamenskoye
616781

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [21]:
# Create a data frame to capture city, latitude, temperature (f), humidity (%), cloudiness (%) and windspeed (mph); check loop totals
city_weather_df = pd.DataFrame({"City": city_name, "Cloudiness": cloudy,"Country": countries, "Date": date, "Humidity": humidity, "Lat": latitude,"Lng": longitude, "Max Temp": max_temp, "Wind Speed": windy})
city_weather_df.count()

City          551
Cloudiness    551
Country       551
Date          551
Humidity      551
Lat           551
Lng           551
Max Temp      551
Wind Speed    551
dtype: int64

In [22]:
city_weather_df = pd.DataFrame({"City": city_name, "Cloudiness": cloudy,"Country": countries, "Date": date, "Humidity": humidity, "Lat": latitude,"Lng": longitude, "Max Temp": max_temp, "Wind Speed": windy})
city_weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Arkhangelsk,75,RU,1568652847,87,64.54,40.54,46.40,2.24
1,Banjar,100,ID,1568652848,91,-7.37,108.54,71.95,1.68
2,Rikitea,0,PF,1568652848,69,-23.12,-134.97,67.99,20.07
3,Mar del Plata,99,AR,1568652848,32,-46.43,-67.52,58.09,30.24
4,Mataura,97,NZ,1568652848,88,-46.19,168.86,39.99,7.00


In [30]:
#export dataframe to csv
city_weather_df = city_weather_df.dropna()
city_weather_df.to_csv('../data/weather_data_csv.csv')

In [29]:
#export dataframe to html
city_weather_df.to_html(buf=None, columns=None, col_space=None, header=True, index=True, na_rep='NaN', formatters=None, float_format=None, sparsify=None, index_names=True, justify=None, bold_rows=True, classes=None, escape=True, max_rows=None, max_cols=None, show_dimensions=False, notebook=False)
city_weather_df.to_html('../data/weather_data_html.html')

# Plotting the Data
#Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
#Save the plotted figures as .pngs.

In [ ]:
#Your objective is to build a series of scatter plots to showcase the following relationships

#assign date and time to scatterplot analyses
import time
here_and_now = time.ctime(today_date)
print(here_and_now)

# Latitude vs. Temperature Plot

In [ ]:
#Temperature (F) vs. Latitude
plt.scatter(latitude, max_temp, marker="^", facecolors="red", alpha=0.75)
plt.grid(linestyle='-')
x_lim = 100
plt.title("City Latitude Vs. Max Temperature" + " " +"("+ here_and_now+")")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")

#save plot to a .png file
plt.savefig("../results/City_latitude_Vs_Max_Temperature.png")
plt.show()

In [ ]:
#save above plot to a .png file
plt.savefig("../Results/City_latitude_Vs_Max_Temperature.png")
#have to run plt.show() again or image will not render in .png
plt.show()

# Latitude vs. Humidity Plot

In [ ]:
#Humidity (%) vs. Latitude
plt.scatter(latitude, humidity, marker="P", facecolors="orange", alpha=0.75)
plt.grid(linestyle='-')
x_lim = 100
plt.title("City Latitude Vs. Humidity" + " " +"("+ here_and_now+")")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")

plt.savefig("../results/City_latitude_Vs_Humidity.png")
plt.show()

# Latitude vs. Cloudiness Plot

In [ ]:
#Cloudiness (%) vs. Latitude
plt.scatter(latitude, cloudy, marker="*", facecolors="aqua", alpha=0.75)
plt.grid(linestyle='-')
x_lim = 100
plt.title("City Latitude Vs. Cloudiness" + " " +"("+ here_and_now+")")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")

plt.savefig("../results/City_latitude_Vs_Cloudiness.png")
plt.show()

# Latitude vs. Wind Speed Plot

In [ ]:
#Wind Speed (mph) vs. Latitude
plt.scatter(latitude, windy, marker=">", facecolors="teal", alpha=0.75)
plt.grid(linestyle='-')
x_lim = 100
plt.title("City Latitude Vs. Wind Speed" + " " +"("+ here_and_now+")")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")

plt.savefig("../results/City_latitude_Vs_Windspeed.png")
plt.show()

# Findings

In [ ]:
#You must include a written description of three observable trends based on the data.

--Temperature is pretty dramatically higher near and just above the equator, but it seems others of these are less significant an will change each time the api is called and the code run.